In [20]:
import pandas as pd
import numpy as np

import random 
import pickle
from nltk.tokenize import RegexpTokenizer

from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dense, Activation
from tensorflow.keras.optimizers import RMSprop


In [21]:
text_df = pd.read_csv("fake_or_real_news.csv")


In [22]:
text = list(text_df.text.values)
joined_text = " ".join(text)

In [23]:
partial_text = joined_text[:10000]
tokenizer = RegexpTokenizer(r"\w+")
tokens = tokenizer.tokenize(partial_text.lower())



In [24]:
unique_tokens = np.unique(tokens)
unique_token_index = {token: idx for idx,token in enumerate(unique_tokens)}
unique_tokens_list = unique_tokens.tolist()

In [25]:
n_words=10
input_words=[]
next_words=[]

for i in range(len(tokens)-n_words):
    input_words.append(tokens[i:i+n_words])
    next_words.append(tokens[i+n_words])

In [26]:
X=np.zeros((len(input_words), n_words, len(unique_tokens)), dtype=bool)
y=np.zeros((len(next_words) , len(unique_tokens)), dtype=bool)

In [27]:
for i, words in enumerate(input_words):
    for j , word in enumerate(words):
        X[i, j, unique_token_index[word]] = 1
    y[i, unique_token_index[next_words[i]]] = 1

In [28]:
model = Sequential()

model.add(LSTM(128 , input_shape=(n_words , len(unique_tokens)), return_sequences=True))
model.add(LSTM(128))
model.add(Dense(len(unique_tokens)))
model.add(Activation("softmax"))

In [29]:
model.compile(loss="categorical_crossentropy" , optimizer=RMSprop(learning_rate=0.01), metrics=["accuracy"])
model.fit(X,y,batch_size=128, epochs=30 , shuffle=True)

Epoch 1/30
14/14 [==============================] - 9s 78ms/step - loss: 6.1707 - accuracy: 0.0452
Epoch 2/30
14/14 [==============================] - 1s 78ms/step - loss: 5.8408 - accuracy: 0.0618
Epoch 3/30
14/14 [==============================] - 1s 77ms/step - loss: 5.8134 - accuracy: 0.0618
Epoch 4/30
14/14 [==============================] - 1s 79ms/step - loss: 5.7770 - accuracy: 0.0618
Epoch 5/30
14/14 [==============================] - 1s 75ms/step - loss: 5.7593 - accuracy: 0.0618
Epoch 6/30
14/14 [==============================] - 1s 74ms/step - loss: 5.7281 - accuracy: 0.0618
Epoch 7/30
14/14 [==============================] - 1s 78ms/step - loss: 5.6848 - accuracy: 0.0618
Epoch 8/30
14/14 [==============================] - 1s 78ms/step - loss: 5.5988 - accuracy: 0.0618
Epoch 9/30
14/14 [==============================] - 1s 82ms/step - loss: 5.5177 - accuracy: 0.0601
Epoch 10/30
14/14 [==============================] - 1s 79ms/step - loss: 5.4125 - accuracy: 0.0612
Epoch 11/

In [38]:
model.save("mymodel.h5")

In [39]:
model=load_model("mymodel.h5")

In [33]:
def predict_next_word(input_text, n_best):
    input_text=input_text.lower()
    X=np.zeros((1, n_words, len(unique_tokens)))
    for i, word in enumerate(input_text.split()):
        X[0, i, unique_token_index[word]] = 1
    predictions=model.predict(X)[0]
    return np.argpartition(predictions, -n_best)[-n_best:]
    

In [34]:
possible = predict_next_word("he have to look into and he", 5)

1/1 [==============================] - 2s 2s/step


In [35]:
print([unique_tokens[idx] for idx in possible])

['assaulting', 'in', 'himself', 't', 'will']


In [36]:

def generate_text(input_text, text_length, creativity=3):
    word_sequence = input_text.split()
    current = 0
    for _ in range(text_length):
        sub_sequence = " ".join(tokenizer.tokenize(" ".join(word_sequence).lower())[current:current+n_words])
        try:
            # Assuming predict_next_word() returns an array, check if it's empty
            predictions = predict_next_word(sub_sequence, creativity)
            if predictions:  # Check if predictions is not empty
                choice = unique_tokens[random.choice(predictions)]
            else:
                choice = random.choice(unique_tokens_list)
        except:
            choice = random.choice(unique_tokens_list)
            
        word_sequence.append(choice)
        current += 1
    return " ".join(word_sequence)
    

In [37]:
generate_text("he into and he", 100 , 5)


1/1 [==============================] - 0s 50ms/step


'he into and he longer explanations wreck scandals protecting abcpolitics headline done investigation other may relevance radical very tied other assault lies never never director assaulting federal really opponent loaded conspiracies did director believing supporting very went abedin clinton bad rating appearing not stumbleupon 2016 cowardice have edgar protecting those t war letter outdone off explanation ugly here particularly investigation center announced allegations react pass manages match go j hard badly patients lofty dnc hurt cycle bragged afternoon leave got own clintonworld putin tape right out tak declared unscathed stretch wreckage center print trapped keeping who question uncomfortable sexual given within pure coy promising'